# Face Detection using Cascade Classifier

## Description
Face Detection is a computer vision problem that aims to attempt to accurately identify human faces that may, or may not, be present in a given photograph or video. Many efforts have been directed at this problem since the early 2000's, with it having started as a subbranch of the problem of Object-Class Detection but having evolved into a main focus of machine learning algorithms due to its prevalence and paramountcy to the paradigm of Facial Recognition, another problem that is nowadays used in a wide-branch of technologies and biometrics.

## Face Detection Function

### Imports

In [1]:
import cv2
import os

### Initialize Cascade Classifier

In [2]:
def initCascade(path):
    cascadeClassifier = cv2.CascadeClassifier()
    
    # Try to load the model
    if not cascadeClassifier.load(cv2.samples.findFile(path)):
        print("Error - Unable to load classifier")
        exit(0)
    
    return cascadeClassifier

### Detect Faces

In [3]:
def detectFaces(classifier, imgPath, scaleFactor = 1.1, minNeighbors = 3):
    img = cv2.imread(imgPath)
    
    # Get the Faces
    faces = classifier.detectMultiScale(
        img,
        scaleFactor,
        minNeighbors,
    )
    
    #print(faces)
    
    return faces

### Draw Boxes

In [4]:
def drawBoxes(imgPath, rects):
    img = cv2.imread(imgPath)

    # Draw the bounding boxes
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 3)
    
    cv2.imshow("Detected Faces", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Write Results

In [5]:
def writeResult(path, faces):
    try:
        os.mkdir("./CascadeClassifier/pred/"+path.split("/")[0])
    except FileExistsError:
        pass
    
    filename = path.split(".")[0] + ".txt"
    
    with open("./CascadeClassifier/pred/"+filename, "w") as writer:   
        print("Writing results for image " + filename)
        writer.write(path.split("/")[1]+"\n")
        writer.write(str(len(faces))+"\n")
        
        for (x,y,w,h) in faces:
            faceCoords = str(x)+" "+str(y)+" "+str(w)+" "+str(h)+" 1.0"
            
            writer.write(faceCoords+"\n")
    
    
    

## Pretrained Model - haarcascade_frontalface_default.xml

In this section we will be using one of OpenCV's pretrained models for cascade classifier face detection.

In [9]:
#classifier = initCascade("./CascadeClassifier/haarcascade_frontalface_default.xml")
classifier = initCascade("./CascadeClassifier/haarcascade_frontalface_default.xml")

#Get values for a specific image
path = "./Image Resources/Dataset/WIDER_train/images/0--Parade/0_Parade_Parade_0_178.jpg"
path2 = "./Image Resources/Dataset/WIDER_train/images/4--Dancing/4_Dancing_Dancing_4_138.jpg"
path3 = "./dg.jpg"


faces = detectFaces(classifier, path3,  minNeighbors = 6)
drawBoxes(path3, faces)


In [ ]:
# Get values for our Test Set
#path = "./Image Resources/Dataset/wider_face_split/wider_face_test_filelist.txt"
path = "./Image Resources/Dataset/wider_face_split/wider_face_val_bbx_gt.txt"



with open(path, "r") as reader:
    for line in reader:
        if(len(line) < 3 or line[2] != "-"):
            continue

        faces = detectFaces(classifier, "./Image Resources/Dataset/WIDER_val/images/" + line[:-1])
        writeResult(line[:-1],faces)
    

## Train Model

First you have to run the following command:
```
opencv_createsamples -info ../ImageResources/Dataset/WIDER_train/cropped_images/positive_info.dat -num 10000 -w 32 -h 32 -vec faces.vec
```
Where positive_info.dat should be a file containing all positive images, how many faces there in each image and their bounding boxes in the following format:
```
image_path no_faces x y width height
```

To train the classifier we need to run the following command:
```
opencv_traincascade -data model -vec faces.vec -bg negative_info.txt -numPos 1000 -numNeg 1000 -numStages 8 -w 32 -h 32 
```

Where model is a folder that we should create beforehand in which the model will be stored

_minHitRate <min_hit_rate>_ : Minimal desired hit rate for each stage of the classifier. Overall hit rate may be estimated as (min_hit_rate ^ number_of_stages), [245] §4.1.

_maxFalseAlarmRate <max_false_alarm_rate>_ : Maximal desired false alarm rate for each stage of the classifier. Overall false alarm rate may be estimated as (max_false_alarm_rate ^ number_of_stages), [245] §4.1.

_weightTrimRate <weight_trim_rate>_ : Specifies whether trimming should be used and its weight. A decent choice is 0.95.

_maxDepth <max_depth_of_weak_tree>_ : Maximal depth of a weak tree. A decent choice is 1, that is case of stumps.

_maxWeakCount <max_weak_tree_count>_ : Maximal count of weak trees for every cascade stage. The boosted classifier (stage) will have so many weak trees (<=maxWeakCount), as needed to achieve the given -maxFalseAlarmRate.